<img align="center" src="images/course.png" width="800">

# 16720 (B)  3D Reconstruction - Assignment 5 - q2
    Instructor: Kris                          TAs: Arka, Jinkun, Rawal, Rohan, Sheng-Yu

In [65]:
# Helper functions for this assignment. DO NOT MODIFY!!!
"""
Helper functions.
"""

import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import scipy
import nbimporter
from q1 import eightpoint, camera2, _epipoles, toHomogenous

def epipolarMatchGUI(I1, I2, F):
    matplotlib.use('TkAgg')
    e1, e2 = _epipoles(F)

    sy, sx, _ = I2.shape

    f, [ax1, ax2] = plt.subplots(1, 2, figsize=(12, 9))
    ax1.imshow(I1)
    ax1.set_title('Select a point in this image')
    ax1.set_axis_off()
    ax2.imshow(I2)
    ax2.set_title('Verify that the corresponding point \n is on the epipolar line in this image')
    ax2.set_axis_off()

    while True:
        plt.sca(ax1)
        x, y = plt.ginput(1, mouse_stop=2)[0]

        xc = int(x)
        yc = int(y)
        v = np.array([xc, yc, 1])
        l = F.dot(v)
        s = np.sqrt(l[0]**2+l[1]**2)

        if s==0:
            error('Zero line vector in displayEpipolar')

        l = l/s;

        if l[0] != 0:
            ye = sy-1
            ys = 0
            xe = -(l[1] * ye + l[2])/l[0]
            xs = -(l[1] * ys + l[2])/l[0]
        else:
            xe = sx-1
            xs = 0
            ye = -(l[0] * xe + l[2])/l[1]
            ys = -(l[0] * xs + l[2])/l[1]

        # plt.plot(x,y, '*', 'MarkerSize', 6, 'LineWidth', 2);
        ax1.plot(x, y, '*', MarkerSize=6, linewidth=2)
        ax2.plot([xs, xe], [ys, ye], linewidth=2)

        # draw points
        x2, y2 = epipolarCorrespondence(I1, I2, F, xc, yc)
        ax2.plot(x2, y2, 'ro', MarkerSize=8, linewidth=2)
        plt.draw()

def plot_3D(P):
    matplotlib.use('TkAgg')
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(P[:,0], P[:,1], P[:,2])
    while True:
        x, y = plt.ginput(1, mouse_stop=2)[0]
        plt.draw()

        


## Q2: Metric Reconstruction
You will compute the camera matrices and triangulate the 2D points to obtain the 3D scene structure. To obtain the Euclidean scene structure, first convert the fundamental matrix $\textbf{F}$ to an essential matrix $\textbf{E}$. Examine the lecture notes and the textbook to find out how to do this when the internal camera calibration matrices $\textbf{K}_1$ and $\textbf{K}_2$ are known; these are provided in `data/intrinsics.npz`.

### Q2.1: Essential Matrix (5 pt implementation)
Write a function to compute the essential matrix $\textbf{E}$ given $\textbf{F}$, $\textbf{K}_1$ and $\textbf{K}_2$ with the signature:

```
E = essentialMatrix(F, K1, K2)
```



In [66]:
def essentialMatrix(F, K1, K2):
    '''
    Q1.1: Compute the essential matrix E given the fundamental matrix and camera intrinsics
        Input:  F, fundamental matrix
                K1, internal camera calibration matrix of camera 1
                K2, internal camera calibration matrix of camera 2
        Output: E, the essential matrix
    '''
    # ----- TODO -----
    # YOUR CODE HERE

    E = K2.T @ F @ K1

    # raise NotImplementedError()
    return E



In [67]:
# Printing out the essential matrix
np.set_printoptions(precision=4, suppress=1)
correspondence = np.load('data/some_corresp.npz') # Loading correspondences
intrinsics = np.load('data/intrinsics.npz') # Loading the intrinscis of the camera
K1, K2 = intrinsics['K1'], intrinsics['K2']
pts1, pts2 = correspondence['pts1'], correspondence['pts2']
im1 = plt.imread('data/im1.png')
im2 = plt.imread('data/im2.png')

F = eightpoint(pts1, pts2, M=np.max([*im1.shape, *im2.shape]))
E = essentialMatrix(F, K1, K2)
print(E)

Optimization terminated successfully.
         Current function value: 0.000107
         Iterations: 8
         Function evaluations: 894
[[  -0.5069   68.6543 -371.9615]
 [  29.7107   -1.5472    9.6823]
 [ 372.9911    2.9855    0.1504]]


In [68]:
# Simple Tests to verify your implmentation:
correspondence = np.load('data/some_corresp.npz') # Loading correspondences
intrinsics = np.load('data/intrinsics.npz') # Loading the intrinscis of the camera
K1, K2 = intrinsics['K1'], intrinsics['K2']
pts1, pts2 = correspondence['pts1'], correspondence['pts2']
F = eightpoint(pts1, pts2, M=np.max([*im1.shape, *im2.shape]))
E = essentialMatrix(F, K1, K2)
assert(np.linalg.matrix_rank(E) == 2)

# Hidden Tests

Optimization terminated successfully.
         Current function value: 0.000107
         Iterations: 8
         Function evaluations: 894


### Q2.2 Triangulation and find M2 (5 pt writeup, 20 pt implementation)
Given an essential matrix, it is possible to retrieve the projective camera matrices $\textbf{M}_1$ and $\textbf{M}_2$ from it.  Assuming $\textbf{M}_1$ is fixed at $[\textbf{I},0]$, $\textbf{M}_2$ can be retrieved up to a scale and four-fold rotation ambiguity. For details on recovering $\textbf{M}_2$, see section 7.2 in Szeliski. We have provided you with the function `camera2` to recover as the four possible $\textbf{M}_2$ matrices given $\textbf{E}$.

**Note:** The $\textbf{M}_1$ and $\textbf{M}_2$ here are projection matrices of the form:
$\textbf{M}_1 = \begin{bmatrix}
\textbf{I} | 0
\end{bmatrix} $ and $\textbf{M}_2 = \begin{bmatrix}
\textbf{R} | \textbf{t}
\end{bmatrix} $.

Using the above, write a function to triangulate a set of 2D coordinates in the image to a set of 3D
points with the signature:
```
    [w, err] = triangulate(C1, pts1, C2, pts2)
```
where `pts1` and `pts2` are the $N \times 2$ matrices with the 2D image coordinates and `w` is an $N \times 3$ matrix with the corresponding 3D points per row.  `C1` and `C2` are the $3 \times 4$ camera matrices. Remember that you will need to multiply the given intrinsics matrices with your solution for the canonical camera matrices to obtain the final camera matrices. Various methods exist for triangulation - probably the most familiar for you is based on least squares (see Szeliski Chapter 7 if you want to learn about other methods):

For each point $i$, we want to solve for 3D coordinates $\textbf{w}_i = \begin{bmatrix}x_i, y_i, z_i\end{bmatrix}^T$, such that when they are projected back to the two images, they are close to the original 2D points. To project the 3D coordinates back to 2D images, we first write $\textbf{w}_i$ in homogeneous coordinates, and compute $\mathbf{C}_1 \tilde{\textbf{w}_i}$ and $\mathbf{C}_2 \tilde{\textbf{w}_i}$ to obtain the 2D homogeneous coordinates projected to camera $1$ and camera $2$, respectively.

For each point $i$, we can write this problem in the following form:
\begin{align}
\mathbf{A}_i w_i = 0
\end{align}
where $\mathbf{A}_i$ is a $4\times 4$ matrix, and $\tilde{\textbf{w}_i}$ is a $4\times 1$ vector of the 3D coordinates in the homogeneous form. Then, you can obtain the homogeneous least-squares solution (discussed in class) to solve for each $\textbf{w}_i$.

Once you have implemented triangulation, you can check the performance by looking at the reprojection error:  $$ \texttt{err} = \sum_i ||\textbf{x}_{1i} - \widehat{\textbf{x}_{1i}}||^2 + ||\textbf{x}_{2i} - \widehat{\textbf{x}_{2i}}||^2$$
where $\widehat{\textbf{x}_{1i}} = Proj(\mathbf{C}_1, \textbf{w}_i)$ and $\widehat{\textbf{x}_{2i}} = Proj(\mathbf{C}_2, \textbf{w}_i)$.


**Note:** `C1` and `C2` here are projection matrices of the form:
$\mathbf{C}_1 = \mathbf{K}_1\mathbf{M}_1 = \mathbf{K}_1 \begin{bmatrix}
\textbf{I} | 0
\end{bmatrix} $ and $\mathbf{C}_2 =  \mathbf{K}_2\mathbf{M}_2 = \mathbf{K}_2 \begin{bmatrix}
\textbf{R} | \textbf{t}
\end{bmatrix}$.

Write a function, 
```
    [M2, C2, P] = findM2()
```
to load the correspondences and use triangulation to find the M2, C2, and P that minimize the reprojection error. 

<span style='color:red'>**Output:**</span> In your write-up: Write down the expression for the matrix $\mathbf{A}_i$

In [69]:
def triangulate(C1, pts1, C2, pts2):
    '''
    Q2.2: Triangulate a set of 2D coordinates in the image to a set of 3D points.
        Input:  C1, the 3x4 camera matrix
                pts1, the Nx2 matrix with the 2D image coordinates per row
                C2, the 3x4 camera matrix
                pts2, the Nx2 matrix with the 2D image coordinates per row
        Output: P, the Nx3 matrix with the corresponding 3D points per row
                err, the reprojection error.
    
    ***
    Hints:
    (1) For every input point, form A using the corresponding points from pts1 & pts2 and C1 & C2
    (2) Solve for the least square solution using np.linalg.svd
    (3) Calculate the reprojection error using the calculated 3D points and C1 & C2 (do not forget to convert from 
        homogeneous coordinates to non-homogeneous ones)
    (4) Keep track of the 3D points and projection error, and continue to next point 
    (5) You do not need to follow the exact procedure above. 
    '''
    # ----- TODO -----
    # YOUR CODE HERE

    p1_C1 = C1[0,:]
    p2_C1 = C1[1,:]
    p3_C1 = C1[2,:]

    p1_C2 = C2[0,:]
    p2_C2 = C2[1,:]
    p3_C2 = C2[2,:]

    N = pts1.shape[0]
    P = np.zeros((N,3))
    err = 0
    for i in range(N):
        x_C1 = pts1[i,0]
        y_C1 = pts1[i,1]
        x_C2 = pts2[i,0]
        y_C2 = pts2[i,1]
        
        A = np.array([(y_C1*p3_C1)-(p2_C1), p1_C1-(x_C1*p3_C1), (y_C2*p3_C2)-p2_C2, p1_C2-(x_C2*p3_C2)])
        # print("A is", A)
        _, _, V = np.linalg.svd(A)

        # print(V.shape)
        X = V[-1,:]
        
        X = np.reshape(X, (4,1))
        X = X/X[-1]
        # print("X shape",X.shape)

        reprojection_C1 = C1 @ X
        x1_reproj = reprojection_C1[0]/reprojection_C1[-1]
        y1_reproj = reprojection_C1[1]/reprojection_C1[-1]
        
        # z1_reproj = C1 * X[2]

        reprojection_C2 = C2 @ X
        x2_reproj = reprojection_C2[0]/reprojection_C2[-1]
        y2_reproj = reprojection_C2[1]/reprojection_C2[-1]
        # z2_reproj = C2 * X[2]

        err += ((x_C1 - x1_reproj)+(y_C1 - y1_reproj))**2 + (((x_C2 - x2_reproj)+(y_C2 - y2_reproj)))**2
        # print(X[:3,0].shape)
        # P[i] = X[:3,0].T
        P[i] = X[:3].T
        # print(P.shape)
    # print(X[:3,0].T)
        


    # raise NotImplementedError()
    return P, err
    

def find_M2(F, pts1, pts2, intrinsics):
    '''
    Q2.2: Function to find the camera2's projective matrix given correspondences
        Input:  F, the pre-computed fundamental matrix
                pts1, the Nx2 matrix with the 2D image coordinates per row
                pts2, the Nx2 matrix with the 2D image coordinates per row
                intrinsics, the intrinsics of the cameras, load from the .npz file
        Output: [M2, C2, P] the computed M2 (3x4) camera projective matrix, C2 (3x4) K2 * M2, and the 3D points P (Nx3)
    
    ***
    Hints:
    (1) Loop through the 'M2s' and use triangulate to calculate the 3D points and projection error. Keep track 
        of the projection error through best_error and retain the best one. 
    (2) Remember to take a look at camera2 to see how to correctly reterive the M2 matrix from 'M2s'. 

    '''
    K1 = intrinsics["K1"]
    K2 = intrinsics["K2"]
    E = essentialMatrix(F, K1, K2)
    M1 = np.hstack((np.identity(3), np.zeros(3)[:,np.newaxis]))
    C1 = K1.dot(M1)
    M2s = camera2(E)
    best_error = np.finfo('float').max
    # ----- TODO -----
    # YOUR CODE HERE
    # last_err = 0
    for i in range(M2s.shape[-1]):
        C2 = K2.dot(M2s[:,:,i])
        P, err = triangulate(C1, pts1, C2, pts2)
        # print("P shape",P.shape)
        # last_err = err
        # for j in range(P.shape[0]):
        if np.all(P[:,-1] > 0):
            # print("I am here")
            if np.sum(err) < best_error:
                best_error = err
                best_M2 = M2s[:,:,i]
                best_C2 = C2
                best_P = P


    # raise NotImplementedError()
    
    M2 = best_M2
    C2 = best_C2
    P = best_P
    print(f"Best Error {best_error}")
    print(M1.shape, C1.shape)
    print(M2.shape, C2.shape)
    # print("P is",P)
    return M2, C2, P

In [70]:
# Running the find_M2 funciton:
np.set_printoptions(precision=4, suppress=1)
correspondence = np.load('data/some_corresp.npz') # Loading correspondences
intrinsics = np.load('data/intrinsics.npz') # Loading the intrinscis of the camera
K1, K2 = intrinsics['K1'], intrinsics['K2']
pts1, pts2 = correspondence['pts1'], correspondence['pts2']
im1, im2 = plt.imread('data/im1.png'), plt.imread('data/im2.png')

F = eightpoint(pts1, pts2, M = np.max([*im1.shape, *im2.shape]))
M2, C2, P = find_M2(F, pts1, pts2, intrinsics)
print(f"M2: {M2}")
print(f"C2 {C2}")

Optimization terminated successfully.
         Current function value: 0.000107
         Iterations: 8
         Function evaluations: 894
Best Error [343.2661]
(3, 4) (3, 4)
(3, 4) (3, 4)
M2: [[ 0.9994  0.0333  0.006  -0.026 ]
 [-0.0337  0.9653  0.2589 -1.    ]
 [ 0.0028 -0.2589  0.9659  0.0796]]
C2 [[ 1520.3898   -27.6331   301.1061   -15.4571]
 [  -50.7634  1409.0488   633.4983 -1506.2442]
 [    0.0028    -0.2589     0.9659     0.0796]]


In [71]:
# Simple Tests to verify your implmentation:
M1 = np.hstack((np.identity(3), np.zeros(3)[:,np.newaxis]))
C1 = K1.dot(M1)
C2 = K2.dot(M2)
P_test, err = triangulate(C1, pts1, C2, pts2)
print('error:', err)
try:
    assert(err < 500)
    print('Test passed!')
except:
    raise AssertionError('Test Failed: error greater than 500!')

error: [343.2661]
Test passed!


In [72]:
# Hidden Tests

### Q2.3 Epipolar Correspondence (5 pt writeup, 10 pt implementation)

You will now create a 3D visualization of the temple images. By treating our
two images as a stereo-pair, we can triangulate corresponding points in each
image, and render their 3D locations.



Implement a function with the
signature:
```
    [x2, y2] = epipolarCorrespondence(im1, im2, F, x1, y1)
```

This function takes in the $x$ and $y$ coordinates of a pixel on `im1` and your fundamental matrix $\textbf{F}$, and returns the coordinates of the pixel on `im2` which correspond to the input point. The match is obtained by computing the similarity of a small window around the $(x_1, y_1)$ coordinates in `im1` to various windows around possible matches in the `im2` and returning the closest.

Instead of searching for the matching point at every possible location in `im2`, we can use $\textbf{F}$ and simply search over the set of pixels that lie along the epipolar line (recall that the epipolar line passes through a single
point in `im2`  which corresponds to the point $(x_1, y_1)$ in `im1`!.

There are various possible ways to compute the window similarity. For this assignment, simple methods such as the Euclidean or Manhattan distances between the intensity of the pixels should suffice.  See Szeliski Chapter 11, on stereo matching, for a brief overview of these and other methods.


**Implementation hints:**
- Experiments with various window sizes.
- It may help to use a Gaussian weighting of the window, so that the center
  has greater influence than the periphery.
- Since the two images only differ by a small amount, it might be beneficial to consider matches for which the distance from $(x_1, y_1)$ to $(x_2, y_2)$ is small.

To help you test your `epipolarCorrespondence`, we have included a helper function `epipolarMatchGUI`, which takes in two images the fundamental matrix.

This GUI allows you to click on a point in `im1`, and will use your function to display the corresponding point in `im2`. See:

<img align="center" src="images/q3gui_disp_arun.png" width="800">

It's not necessary for your matcher to get **every** possible point right, but it should get easy points (such as those with distinctive corner-like windows).

It should also be good enough to render an intelligible representation in the next question.

<span style='color:red'>**Output:**</span> In your write-up, include a screenshot of `epipolarMatchGUI`
with some detected correspondences.


In [73]:
def epipolarCorrespondence(im1, im2, F, x1, y1):
    '''
    Q2.3 3D visualization of the temple images.
        Input:  im1, the first image
                im2, the second image
                F, the fundamental matrix
                x1, x-coordinates of a pixel on im1
                y1, y-coordinates of a pixel on im1
        Output: x2, x-coordinates of the pixel on im2
                y2, y-coordinates of the pixel on im2
    
    ***
    Hints:
    (1) Given input [x1, x2], use the fundamental matrix to recover the corresponding epipolar line on image2
    (2) Search along this line to check nearby pixel intensity (you can define a search window) to 
        find the best matches
    (3) Use guassian weighting to weight the pixel simlairty
    
    '''
    # ----- TODO -----
    # YOUR CODE HERE
    homo_pt = np.array([[x1], [y1], [1]])
    epi_line = F @ homo_pt
    a = epi_line[0,0]
    b = epi_line[1,0]
    c = epi_line[2,0]
    # print(im2[0,:,:])
    window = 15
    img1_intensity = im1[int(y1-(window//2)):int(y1+(window//2)), int(x1-(window//2)):int(x1+(window//2)), :]
    best_error = np.finfo('float').max
    for y in range(int(window//2), im2.shape[0]-int(window//2)):
        # y = i
        # print(y)
        x = -(c/a) - ((b*y)/a)
        img2_intensity = im2[int(y-(window//2)):int(y+(window//2)), int(x-(window//2)):int(x+(window//2)), :]

        diff = np.sum(abs(img1_intensity-img2_intensity))
        # print("diff shape", diff.shape)
        if diff<best_error:
            best_error = diff
            x2 = x
            y2 = y
        
    print("coords",x2,y2)
    
    

    # raise NotImplementedError()
    return x2, y2


In [75]:
# Visualization:
np.set_printoptions(precision=4, suppress=1)
correspondence = np.load('data/some_corresp.npz') # Loading correspondences
intrinsics = np.load('data/intrinsics.npz') # Loading the intrinscis of the camera
K1, K2 = intrinsics['K1'], intrinsics['K2']
pts1, pts2 = correspondence['pts1'], correspondence['pts2']
im1, im2 = plt.imread('data/im1.png'), plt.imread('data/im2.png')

F = eightpoint(pts1, pts2, M=np.max([*im1.shape, *im2.shape]))
# epipolarMatchGUI(im1, im2, F) ## uncomment to visualize, comment out before submitting to gradescope

Optimization terminated successfully.
         Current function value: 0.000107
         Iterations: 8
         Function evaluations: 894


/tmp/ipykernel_3592/3463546744.py:54: MatplotlibDeprecationWarning: Case-insensitive properties were deprecated in 3.3 and support will be removed two minor releases later
  ax1.plot(x, y, '*', MarkerSize=6, linewidth=2)
/tmp/ipykernel_3592/3463546744.py:59: MatplotlibDeprecationWarning: Case-insensitive properties were deprecated in 3.3 and support will be removed two minor releases later
  ax2.plot(x2, y2, 'ro', MarkerSize=8, linewidth=2)


coords 64.19080906815967 178
coords 113.88706423326246 287
coords 166.42606073049532 191
coords 212.87787692780458 169
coords 271.8695431467661 155
coords 364.3570289706422 150
coords 472.877957111953 142
coords 475.50284661773173 272
coords 432.7857554016746 254
coords 366.65939422334225 287
coords 263.16958034644745 271
coords 310.4920286717071 270
coords 279.64938627724564 354
coords 355.14535019780925 354
coords 334.7816836520365 193
coords 512.0259087884015 182
coords 98.95333016875797 224
coords 72.33857090793445 129


TclError: invalid command name "pyimage20"

In [ ]:
# Simple Tests to verify your implmentation:
correspondence = np.load('data/some_corresp.npz') # Loading correspondences
intrinsics = np.load('data/intrinsics.npz') # Loading the intrinscis of the camera
K1, K2 = intrinsics['K1'], intrinsics['K2']
pts1, pts2 = correspondence['pts1'], correspondence['pts2']
im1, im2 = plt.imread('data/im1.png'), plt.imread('data/im2.png')
F = eightpoint(pts1, pts2, M=np.max([*im1.shape, *im2.shape]))
x2, y2 = epipolarCorrespondence(im1, im2, F, 119, 217)

try:
    assert(np.linalg.norm(np.array([x2, y2]) - np.array([118, 181])) < 10)
    print('Test passed!')
except:
    raise AssertionError('Test Failed: error greater than 10!')


Optimization terminated successfully.
         Current function value: 0.000107
         Iterations: 8
         Function evaluations: 894
coords 118.42077998451191 181
Test passed!


In [ ]:
# Hidden Tests

### Q2.4 3D Visualization (3pt writeup, 7 pt implementation) 

Included in this homework  is a file data/templeCoords.npz which contains 288 hand-selected points from `im1` saved in the variables `x1` and `y1`.

Now, we can determine the 3D location of these point correspondences using the `triangulate` function. These 3D point locations can then plotted using the `Matplotlib` package (we have provided the necessary starter code for visualization). An example is shown here: 


|![alt](images/q3a.png) |![alt](images/q3b.png)|
|-|-|
|![alt](images/q3b.png) |![alt](images/q3c.png)|
|-|-|


<span style='color:red'>**Output:**</span> In your write-up: Take a few screenshots of the 3D visualization
so that the outline of the temple is clearly visible.

In [ ]:
def compute3D_pts(pts1, intrinsics, F, im1, im2):
    '''
    Q2.4: Finding the 3D position of given points based on epipolar correspondence and triangulation
        Input:  pts1, chosen points from im1
                intrinsics, the intrinsics dictionary for calling epipolarCorrespondence
                F, the fundamental matrix
                im1, the first image
                im2, the second image
        Output: P (Nx3) the recovered 3D points
    
    ***
    Hints:
    (1) Use epipolarCorrespondence to find the corresponding point for [x1 y1] (find [x2, y2])
    (2) Now you have a set of corresponding points [x1, y1] and [x2, y2], you can compute the M2
        matrix and use triangulate to find the 3D points. 
    (3) Use the function findM2 to find the 3D points P (do not recalculate fundamental matrices)
    (4) As a reference, our solution's bet error is around ~2000 on the 3D points. 
    '''
    x1s_temple, y1s_temple = pts1[:, 0], pts1[:, 1]
    # ----- TODO -----
    # YOUR CODE HERE
    pts2 = np.zeros((len(x1s_temple),2))
    for i in range(len(x1s_temple)):
        pts2[i,0], pts2[i,1] = epipolarCorrespondence(im1, im2, F, x1s_temple[i], y1s_temple[i])
        
    M2, C2, P = find_M2(F, pts1, pts2, intrinsics)
    # P, err = triangulate(C1, pts1, C2, pts2)
    # raise NotImplementedError()
    return P



In [76]:
# Visualization:
correspondence = np.load('data/some_corresp.npz') # Loading correspondences
intrinsics = np.load('data/intrinsics.npz') # Loading the intrinscis of the camera
templeCoords = np.load("data/templeCoords.npz")
pts1, pts2 = correspondence['pts1'], correspondence['pts2']
im1, im2 = plt.imread('data/im1.png'), plt.imread('data/im2.png')

F = eightpoint(pts1, pts2, M=np.max([*im1.shape, *im2.shape]))

pts1 = np.hstack([templeCoords["x1"], templeCoords["y1"]])
P = compute3D_pts(pts1, intrinsics, F, im1, im2)

# plot_3D(P) ## uncomment to visualize, comment out before submitting to gradescope


Optimization terminated successfully.
         Current function value: 0.000107
         Iterations: 8
         Function evaluations: 894
coords 54.36078801610039 177
coords 60.41616311935435 166
coords 62.42698187233093 153
coords 64.48366790982327 141
coords 65.47908930336567 124
coords 76.33191354275938 121
coords 95.1834663630598 264
coords 107.04105998846113 134
coords 117.881098964125 132
coords 133.49768188064274 125
coords 145.3932581674002 134
coords 156.2374956963213 137
coords 168.0591709277888 137
coords 180.85976805219906 139
coords 189.63811521674398 133
coords 202.4473180084189 134
coords 214.2354956937473 134
coords 224.14395788056814 138
coords 234.89381035982697 135
coords 244.84943363147926 140
coords 256.8388725790023 147
coords 272.65881191433687 149
coords 285.4380013960911 148
coords 297.3607848994675 151
coords 311.1343209669359 150
coords 324.97137663034766 151
coords 335.81504766657605 151
coords 349.6581066637235 152
coords 364.44676624372335 152
coords 381.3

TclError: invalid command name "pyimage30"

In [ ]:
# Hidden Tests

### Q2.5 Extra Credits: More 3D Visualization (10pts writeup) 

Similar to the Q2.4 visualization of the Temple image, show the 3D reconstruction of any other object using your own images. Feel free to use the images and camera parameters from the [Middlebury multiview dataset](http://vision.middlebury.edu/mview/data/) or other public datasets. Please restrict to only using 2 images and atleast 10 correspondences in both the images.



<span style='color:red'>**Output:**</span> In your write-up: 
- Show the two images of the object from different views.
- Visualize the 2D correspondences on both the images.
- Take a few screenshots of the 3D visualization of the reconstruction.